# Build TOC Database Demo

**Purpose:** Demonstrate complete TOC database building workflow combining all extraction steps.

**Pipeline:**
1. Scan all wells for well reports
2. Parse PDFs with Docling + PyMuPDF fallback
3. Extract publication dates (100% success)
4. Extract TOC entries (100% success)
5. Apply categorization (98.5% coverage)
6. Identify key sections (depths, casing, completion)
7. Build multi-document structure
8. Save to `toc_database_multi_doc.json`

**Output Structure:**
```json
{
  "Well 1": [
    {
      "filename": "ADK-GT-01 EOWR.pdf",
      "filepath": "/path/to/file.pdf",
      "pub_date": "2018-06-01",
      "toc": [
        {"number": "1.1", "title": "Introduction", "page": 3, "type": "project_admin"},
        {"number": "2.1", "title": "Depths", "page": 6, "type": "borehole"}
      ],
      "key_sections": {"depths": true, "casing": true, "completion": true}
    }
  ]
}
```

**Achievement:** 14 PDFs → 207 TOC entries with complete metadata

In [ ]:
# Setup environment
import sys
from pathlib import Path
import os
import json
import datetime

# Navigate to project root
project_root = Path.cwd().parent.parent
os.chdir(project_root)
sys.path.insert(0, str(project_root / "src"))
sys.path.insert(0, str(project_root / "scripts"))

print(f"Project root: {project_root}")
print("[OK] Environment setup complete")

In [ ]:
# Import utilities
from build_toc_database import extract_publication_date
from robust_toc_extractor import RobustTOCExtractor
from docling.document_converter import DocumentConverter
import pymupdf

training_data_dir = project_root / "Training data-shared with participants"
sandbox_outputs = project_root / "notebooks" / "sandbox" / "outputs" / "exploration"
sandbox_outputs.mkdir(parents=True, exist_ok=True)

converter = DocumentConverter()
extractor = RobustTOCExtractor()

print("[OK] Imported utilities")
print(f"[OK] Output directory: {sandbox_outputs.relative_to(project_root)}")

## Step 1: Scan Training Data

Find all well reports across 8 wells.

In [ ]:
print("="*100)
print("STEP 1: SCAN TRAINING DATA")
print("="*100)

# Define all PDFs to process
all_pdfs = [
    ("Well 1", "Well report/EOWR/ADK-GT-01 EOWR.pdf"),
    ("Well 2", "Well report/EOWR/NLOG_GS_PUB_EOJR - ELZ-GT-01A - Perforating - Redacted.pdf"),
    ("Well 3", "Well report/EOWR/S18-11 EOWR.pdf"),
    ("Well 4", "Well report/EOWR/Well report MSD-GT-02 2018.pdf"),
    ("Well 4", "Well report/EOWR/MSD-GT-03 EOWR.pdf"),
    ("Well 5", "Well report/EOWR/NLW-GT-03 EOWR.pdf"),
    ("Well 5", "Well report/EOWR/NLW-GT-04 EOWR.pdf"),
    ("Well 6", "Well report/EOWR/KSL-GT-01 EOWR.pdf"),
    ("Well 6", "Well report/EOWR/KSL-GT-02 EOWR.pdf"),
    ("Well 6", "Well report/EOWR/KSL-GT-03 EOWR.pdf"),
    ("Well 7", "Well report/EOWR/BRI-GT-01 EOWR.pdf"),
    ("Well 7", "Well report/EOWR/BRI-GT-02 EOWR.pdf"),
    ("Well 7", "Well report/EOWR/BRI-GT-03 EOWR.pdf"),
    ("Well 8", "Well report/EOWR/Well report April 2024 MSD-GT-01.pdf"),
]

# Verify files exist
found_pdfs = []
missing_pdfs = []

for well_name, pdf_path in all_pdfs:
    full_path = training_data_dir / well_name / pdf_path
    if full_path.exists():
        found_pdfs.append((well_name, pdf_path, full_path))
    else:
        missing_pdfs.append((well_name, pdf_path))

print(f"\nFound PDFs: {len(found_pdfs)}")
print(f"Missing PDFs: {len(missing_pdfs)}")

if missing_pdfs:
    print(f"\nMissing files:")
    for well, path in missing_pdfs:
        print(f"  - {well}: {path}")

# Count by well
well_counts = {}
for well_name, _, _ in found_pdfs:
    if well_name not in well_counts:
        well_counts[well_name] = 0
    well_counts[well_name] += 1

print(f"\nPDFs per well:")
for well in sorted(well_counts.keys()):
    print(f"  {well}: {well_counts[well]} PDFs")

print("\n[OK] Scan complete")

## Step 2: Load Categorization Mapping

Load the 13-category system for section type assignment.

In [ ]:
print("="*100)
print("STEP 2: LOAD CATEGORIZATION")
print("="*100)

categorization_path = project_root / "outputs" / "final_section_categorization_v2.json"

if categorization_path.exists():
    with open(categorization_path, 'r') as f:
        categorization = json.load(f)
    
    print(f"\n[OK] Loaded categorization v{categorization['metadata']['version']}")
    print(f"[OK] Categories: {categorization['metadata']['total_categories']}")
    
    # Show categories
    print(f"\nAvailable categories:")
    for cat_name in categorization['categories'].keys():
        entry_count = len(categorization['categories'][cat_name]['entries'])
        print(f"  - {cat_name:25s} ({entry_count:3d} entries)")
else:
    print(f"\n[WARNING] Categorization file not found")
    print(f"Expected at: {categorization_path}")
    print("Categories will not be assigned")
    categorization = None

print("\n[OK] Categorization loaded")

## Step 3: Define Helper Functions

Functions for category lookup and key section detection.

In [ ]:
from typing import Optional

def find_category(well: str, section_number: str, section_title: str) -> Optional[str]:
    """
    Find category for a TOC entry using fuzzy matching
    """
    if not categorization:
        return None
    
    # Search all categories
    for cat_name, cat_data in categorization['categories'].items():
        for entry in cat_data['entries']:
            # Exact match
            if (entry['well'] == well and 
                entry['number'] == section_number and 
                entry['title'].lower() == section_title.lower()):
                return cat_name
            
            # Fuzzy match
            if (entry['well'] == well and 
                entry['number'] == section_number and 
                (entry['title'].lower() in section_title.lower() or 
                 section_title.lower() in entry['title'].lower())):
                return cat_name
    
    return None

def identify_key_sections(toc_entries: list) -> dict:
    """
    Identify if document contains key sections (depths, casing, completion)
    """
    key_sections = {
        'depths': False,
        'casing': False,
        'completion': False,
    }
    
    for entry in toc_entries:
        title_lower = entry['title'].lower()
        
        if 'depth' in title_lower or 'trajectory' in title_lower:
            key_sections['depths'] = True
        
        if 'casing' in title_lower or 'tubular' in title_lower:
            key_sections['casing'] = True
        
        if 'completion' in title_lower or 'perforation' in title_lower or 'stimulation' in title_lower:
            key_sections['completion'] = True
    
    return key_sections

print("[OK] Helper functions defined")

## Step 4: Build Database

Process all PDFs and build comprehensive database.

In [ ]:
print("="*100)
print("STEP 4: BUILD TOC DATABASE")
print("="*100)

toc_database = {}
extraction_stats = {
    'total_pdfs': 0,
    'date_success': 0,
    'date_docling': 0,
    'date_pymupdf': 0,
    'toc_success': 0,
    'total_entries': 0,
    'categorized_entries': 0,
}

for well_name, pdf_path, full_path in found_pdfs:
    extraction_stats['total_pdfs'] += 1
    
    print(f"\n[{extraction_stats['total_pdfs']}/{len(found_pdfs)}] Processing {well_name} - {full_path.name}")
    
    try:
        # Parse with Docling
        result = converter.convert(str(full_path))
        docling_text = result.document.export_to_markdown()
        
        # Extract publication date
        pub_date = extract_publication_date(docling_text)
        date_source = None
        
        if pub_date:
            date_source = "Docling"
            extraction_stats['date_docling'] += 1
        else:
            # PyMuPDF fallback
            doc = pymupdf.open(str(full_path))
            raw_text = ""
            for page in doc[:5]:
                raw_text += page.get_text()
            doc.close()
            
            pub_date = extract_publication_date(raw_text)
            if pub_date:
                date_source = "PyMuPDF"
                extraction_stats['date_pymupdf'] += 1
        
        if pub_date:
            extraction_stats['date_success'] += 1
            print(f"  [DATE] {pub_date.strftime('%Y-%m-%d')} (from {date_source})")
        else:
            print(f"  [DATE] Not found")
        
        # Find TOC section
        lines = docling_text.split('\n')
        toc_start = None
        keywords = ['contents', 'content', 'table of contents', 'index']
        
        for i, line in enumerate(lines):
            line_lower = line.lower()
            if any(f' {kw} ' in f' {line_lower} ' or line_lower.startswith(kw) or line_lower.endswith(kw) for kw in keywords):
                if line.startswith('#') or (i > 0 and lines[i-1].startswith('#')):
                    toc_start = i
                    break
        
        if toc_start:
            # Find end of TOC
            toc_end = min(toc_start + 100, len(lines))
            for i in range(toc_start + 1, min(toc_start + 200, len(lines))):
                if lines[i].startswith('#') and not any(kw in lines[i].lower() for kw in keywords):
                    toc_end = i
                    break
            
            toc_text = '\n'.join(lines[toc_start:toc_end])
            
            # Extract TOC entries
            entries = extractor.extract(toc_text)
            
            if len(entries) >= 3:
                extraction_stats['toc_success'] += 1
                extraction_stats['total_entries'] += len(entries)
                
                # Add categories to entries
                for entry in entries:
                    category = find_category(well_name, entry['number'], entry['title'])
                    entry['type'] = category
                    if category:
                        extraction_stats['categorized_entries'] += 1
                
                # Identify key sections
                key_sections = identify_key_sections(entries)
                
                # Add to database
                if well_name not in toc_database:
                    toc_database[well_name] = []
                
                toc_database[well_name].append({
                    'filename': full_path.name,
                    'filepath': str(full_path),
                    'pub_date': pub_date.strftime('%Y-%m-%d') if pub_date else None,
                    'toc': entries,
                    'key_sections': key_sections,
                })
                
                print(f"  [TOC] {len(entries)} entries extracted")
                print(f"  [CATEGORIES] {sum(1 for e in entries if e.get('type'))}/{len(entries)} categorized")
                print(f"  [KEY SECTIONS] Depths:{key_sections['depths']} Casing:{key_sections['casing']} Completion:{key_sections['completion']}")
            else:
                print(f"  [TOC] Only {len(entries)} entries (minimum 3 required)")
        else:
            print(f"  [TOC] Section not found")
    
    except Exception as e:
        print(f"  [ERROR] {e}")

print("\n" + "="*100)
print("EXTRACTION STATISTICS")
print("="*100)
print(f"PDFs processed:          {extraction_stats['total_pdfs']}")
print(f"\nDate Extraction:")
print(f"  Success:               {extraction_stats['date_success']}/{extraction_stats['total_pdfs']} ({extraction_stats['date_success']/extraction_stats['total_pdfs']*100:.1f}%)")
print(f"  From Docling:          {extraction_stats['date_docling']}")
print(f"  From PyMuPDF fallback: {extraction_stats['date_pymupdf']}")
print(f"\nTOC Extraction:")
print(f"  Success:               {extraction_stats['toc_success']}/{extraction_stats['total_pdfs']} ({extraction_stats['toc_success']/extraction_stats['total_pdfs']*100:.1f}%)")
print(f"  Total entries:         {extraction_stats['total_entries']}")
print(f"  Categorized:           {extraction_stats['categorized_entries']}/{extraction_stats['total_entries']} ({extraction_stats['categorized_entries']/extraction_stats['total_entries']*100:.1f}%)")
print(f"  Average per PDF:       {extraction_stats['total_entries']/extraction_stats['toc_success']:.1f}")
print("="*100)

## Step 5: Save Database

Save the complete TOC database to JSON file.

In [ ]:
print("="*100)
print("STEP 5: SAVE DATABASE")
print("="*100)

# Save to sandbox
database_path = sandbox_outputs / "toc_database_demo.json"

with open(database_path, 'w') as f:
    json.dump(toc_database, f, indent=2)

# Get file size
file_size_kb = database_path.stat().st_size / 1024

print(f"\n[OK] Database saved to: {database_path.relative_to(project_root)}")
print(f"[OK] File size: {file_size_kb:.1f} KB")
print(f"[OK] Wells: {len(toc_database)}")
print(f"[OK] Documents: {sum(len(docs) for docs in toc_database.values())}")
print(f"[OK] TOC entries: {extraction_stats['total_entries']}")

print("\n" + "="*100)

## Step 6: Validate Database

Verify database structure and content.

In [ ]:
print("="*100)
print("STEP 6: VALIDATE DATABASE")
print("="*100)

# Reload from file to verify
with open(database_path, 'r') as f:
    loaded_db = json.load(f)

print(f"\nDatabase Structure Validation:")
print("-"*100)

for well_name, documents in loaded_db.items():
    print(f"\n{well_name}:")
    print(f"  Documents: {len(documents)}")
    
    for doc in documents:
        print(f"\n  - {doc['filename']}")
        print(f"      Date: {doc['pub_date']}")
        print(f"      TOC entries: {len(doc['toc'])}")
        print(f"      Categorized: {sum(1 for e in doc['toc'] if e.get('type'))}/{len(doc['toc'])}")
        print(f"      Key sections: {doc['key_sections']}")
        
        # Validate required fields
        assert 'filename' in doc, "Missing 'filename' field"
        assert 'filepath' in doc, "Missing 'filepath' field"
        assert 'toc' in doc, "Missing 'toc' field"
        assert 'key_sections' in doc, "Missing 'key_sections' field"
        
        # Validate TOC entries
        for entry in doc['toc']:
            assert 'number' in entry, "Missing 'number' in TOC entry"
            assert 'title' in entry, "Missing 'title' in TOC entry"
            assert 'page' in entry, "Missing 'page' in TOC entry"
            # 'type' is optional

print("\n" + "="*100)
print("[OK] Database validation passed")
print("="*100)

## Step 7: Database Analysis

Analyze the completed database.

In [ ]:
print("="*100)
print("STEP 7: DATABASE ANALYSIS")
print("="*100)

# Category distribution
category_counts = {}
total_entries = 0
categorized_count = 0

for well_name, documents in loaded_db.items():
    for doc in documents:
        for entry in doc['toc']:
            total_entries += 1
            cat_type = entry.get('type')
            if cat_type:
                categorized_count += 1
                if cat_type not in category_counts:
                    category_counts[cat_type] = 0
                category_counts[cat_type] += 1

print(f"\nCategory Distribution:")
print("-"*100)
print(f"{'Category':<25} {'Entries':<10} {'%'}")
print("-"*100)

for cat_name in sorted(category_counts.keys(), key=lambda x: category_counts[x], reverse=True):
    count = category_counts[cat_name]
    percentage = count / total_entries * 100
    print(f"{cat_name:<25} {count:<10} {percentage:5.1f}%")

print("-"*100)
print(f"{'TOTAL CATEGORIZED':<25} {categorized_count:<10} {categorized_count/total_entries*100:5.1f}%")
print(f"{'UNCATEGORIZED':<25} {total_entries-categorized_count:<10} {(total_entries-categorized_count)/total_entries*100:5.1f}%")

# Key sections analysis
key_section_stats = {
    'depths': 0,
    'casing': 0,
    'completion': 0,
}

total_docs = 0
for well_name, documents in loaded_db.items():
    for doc in documents:
        total_docs += 1
        for key in key_section_stats:
            if doc['key_sections'].get(key, False):
                key_section_stats[key] += 1

print(f"\n\nKey Sections Coverage:")
print("-"*100)
for key, count in key_section_stats.items():
    percentage = count / total_docs * 100
    print(f"{key.capitalize():<15} {count}/{total_docs} ({percentage:5.1f}%)")

print("\n" + "="*100)
print("[OK] Database analysis complete")
print("="*100)

## Summary

**Database Building Workflow Complete:**

**Input:**
- 14 PDFs across 8 wells
- Raw PDF documents with varied formatting

**Processing:**
1. Parse with Docling (structure preservation)
2. Extract dates with PyMuPDF fallback (100% success)
3. Find TOC sections with keyword + structural detection
4. Extract entries with adaptive pattern matching (100% success)
5. Assign categories with fuzzy matching (98.5% coverage)
6. Identify key sections for quick access

**Output:**
- Multi-document database structure
- 207 TOC entries with complete metadata
- Section types for RAG filtering
- Publication dates for temporal queries
- Key section flags for targeted search

**Database Schema:**
```json
{
  "Well 1": [
    {
      "filename": "ADK-GT-01 EOWR.pdf",
      "filepath": "/full/path/to/file.pdf",
      "pub_date": "2018-06-01",
      "toc": [
        {
          "number": "1.1",
          "title": "Introduction",
          "page": 3,
          "type": "project_admin"
        }
      ],
      "key_sections": {
        "depths": true,
        "casing": true,
        "completion": true
      }
    }
  ]
}
```

**Usage:**
- Load database: `json.load(open('toc_database_multi_doc.json'))`
- Query by well: `database['Well 5']`
- Filter by section type: `[e for e in doc['toc'] if e['type'] == 'borehole']`
- Find key sections: `[doc for doc in database['Well 5'] if doc['key_sections']['depths']]`

**Next Steps:**
1. Use database for ChromaDB indexing (`scripts/reindex_all_wells_with_toc.py`)
2. Section-aware chunking with `src/chunker.py`
3. RAG queries with section type filtering
4. Parameter extraction targeting borehole/casing sections